In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
# Cài đặt Unsloth và các thư viện cần thiết
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers
from unsloth import FastLanguageModel
import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-vs0imfza/unsloth_ad30e328bc054220945d725c91aeb491
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-vs0imfza/unsloth_ad30e328bc054220945d725c91aeb491
  Resolved https://github.com/unslothai/unsloth.git to commit a2d7811fe8283475f50c88645fb5d124dedb714c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.24.0-py3-none-any.whl (423 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.8.6
    Uninstalling trl-0.8.6:
      Successfully uninstalled trl-0.8.6
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl 

In [ ]:
model_path = "/content/drive/MyDrive/NLP/trained_model"
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

# 3. Định nghĩa Prompt Template (Alpaca Format)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Bạn là một chuyên gia AI về phân tích cảm xúc khách hàng (ABSA). Nhiệm vụ của bạn là đọc bình luận và trích xuất các khía cạnh (Aspect) cùng cảm xúc (Sentiment) và nội dung tương ứng (Span).

DANH SÁCH NHÃN:
- Aspect: [SCREEN, CAMERA, FEATURES, BATTERY, PERFORMANCE, STORAGE, DESIGN, PRICE, GENERAL, SER&ACC]
- Sentiment: [POSITIVE, NEGATIVE, NEUTRAL]

HƯỚNG DẪN GÁN NHÃN QUAN TRỌNG:
1. SER&ACC (Dịch vụ & Phụ kiện): Bao gồm CSKH, bảo hành, giao hàng, đóng gói, và các phụ kiện đi kèm (sạc, cáp, tai nghe...).
   - Ví dụ: "Giao hàng nhanh" -> SER&ACC#POSITIVE. "Sạc lỏng lẻo" -> SER&ACC#NEGATIVE.
2. FEATURES (Tính năng): Các tính năng cụ thể như Vân tay, FaceID, Wifi, 4G, Bluetooth, Sim, Chống nước...
   - Ví dụ: "Vân tay nhạy" -> FEATURES#POSITIVE. "Bắt wifi kém" -> FEATURES#NEGATIVE.
3. QUY TẮC NHÃN 'GENERAL' (Tổng quan):
   - GENERAL#POSITIVE: Nếu (Tổng số nhãn Positive khác) - (Tổng số nhãn Negative khác) >= 2.
   - GENERAL#NEGATIVE: Nếu (Tổng số nhãn Negative khác) - (Tổng số nhãn Positive khác) >= 2.
   - GENERAL#NEUTRAL: Các trường hợp còn lại (chênh lệch <= 1) hoặc khi khách hàng nhận xét chung chung "Máy ổn", "Tạm được".

ĐỊNH DẠNG OUTPUT:
Trả về JSON list chứa các object: {{"aspect": "...", "sentiment": "...", "span": "..."}}. Nếu không có thông tin, trả về [].
### Input:
{}

### Output:
"""

while True:
    input_text = input("\n>> Input: ")

    if input_text.lower() in ['exit', 'quit', 'q']:
        break

    if not input_text.strip():
        continue

    # 3. Tokenize & Generate
    inputs = tokenizer(
        [alpaca_prompt.format(input_text)],
        return_tensors = "pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens = 512,
        use_cache = True,
        temperature = 0.1,
        top_p = 0.9,
    )

    # 4. Decode & In kết quả
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    generated_json = response.split("### Output:")[-1].strip()

    print("MODEL OUTPUT:")
    print(generated_json)

==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

>> Input: Thiết kế vuông vức nhìn sang chảnh thật đấy, cầm rất đầm tay nhưng mà cụm camera lồi quá, đặt lên bàn cứ bị bập bênh khó chịu.
MODEL OUTPUT:
[{"aspect": "DESIGN", "sentiment": "POSITIVE", "span": "Thiết kế vuông vức nhìn sang chảnh thật đấy"}, {"aspect": "CAMERA", "sentiment": "NEGATIVE", "span": "cụm camera lồi quá, đặt lên bàn cứ bị bập bênh khó chịu"}]

>> Input: Sáng rút sạc 100% đi làm, mới lướt toptop với check mail tí xíu mà trưa đã phải cắm dự phòng rồi, nản thực sự.
MODEL OUTPUT:
[{"aspect": "BATTERY", "sentiment": "N